In [3]:
from transformers import pipeline

In [4]:
classifier = pipeline('sentiment-analysis')
classifier('We are very happy to show you the 🤗 Transformers library.')

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# alternative to instantiate model and tokenizer without the automagic of happyface
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

Customizing the model using the config, but the model is from scratch - not pretrained

In [30]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification
config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4*512)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification(config)

In [8]:
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(inputs)

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [10]:
pt_batch = tokenizer(
     ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
     padding=True,
     truncation=True,
     max_length=512,
     return_tensors="pt"
)

for key, value in pt_batch.items():
     print(f"{key}: {value.numpy().tolist()}")

input_ids: [[101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


In [12]:
pt_outputs = pt_model(**pt_batch)
print(pt_outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.0833,  4.3364],
        [ 0.0818, -0.0418]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


In [14]:
import torch.nn.functional as F
pt_predictions = F.softmax(pt_outputs[0], dim=-1)
print(pt_predictions)

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward>)


In [15]:
import torch
pt_outputs = pt_model(**pt_batch, labels = torch.tensor([1, 0]))

In [25]:
# saving the model

save_directory = "./test"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

# loading the model

from transformers import AutoModel
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModel.from_pretrained(save_directory, from_tf=False)



In [27]:
pt_outputs = pt_model(**pt_batch, output_hidden_states=True, output_attentions=True)
all_hidden_states, all_attentions = pt_outputs[-2:]
print(all_hidden_states)
print(all_attentions)

(tensor([[[ 0.3549, -0.1386, -0.2253,  ...,  0.1536,  0.0748,  0.1310],
         [-0.5773,  0.6791, -0.9738,  ...,  0.8805,  1.1044, -0.7628],
         [-0.3451, -0.2094,  0.5709,  ...,  0.3208,  0.0853,  0.4575],
         ...,
         [ 0.4431,  0.0931, -0.1034,  ..., -0.7737,  0.0813,  0.0728],
         [-0.5605,  0.1081,  0.1229,  ...,  0.4519,  0.2104,  0.2970],
         [-0.6116,  0.0156, -0.0555,  ..., -0.1736,  0.1933, -0.0021]],

        [[ 0.3549, -0.1386, -0.2253,  ...,  0.1536,  0.0748,  0.1310],
         [-0.5773,  0.6791, -0.9738,  ...,  0.8805,  1.1044, -0.7628],
         [-0.7195, -0.0363, -0.6576,  ...,  0.4434,  0.3358, -0.9249],
         ...,
         [ 0.0073, -0.5248,  0.0049,  ...,  0.2801, -0.2253,  0.1293],
         [-0.0790, -0.5581,  0.2347,  ...,  0.2370, -0.5104,  0.0770],
         [-0.0958, -0.5744,  0.2631,  ...,  0.2453, -0.3293,  0.1269]]],
       grad_fn=<NativeLayerNormBackward>), tensor([[[ 5.0274e-02,  1.2093e-02, -1.1208e-01,  ...,  6.2100e-02,
    

## Question Answering example

In [31]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch."""

questions = [
     "How many pretrained models are available in 🤗 Transformers?",
     "What does 🤗 Transformers provide?",
     "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
     inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
     input_ids = inputs["input_ids"].tolist()[0]

     text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
     outputs = model(**inputs)
     answer_start_scores = outputs.start_logits
     answer_end_scores = outputs.end_logits

     answer_start = torch.argmax(
         answer_start_scores
     )  # Get the most likely beginning of answer with the argmax of the score
     answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

     answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

     print(f"Question: {question}")
     print(f"Answer: {answer}")


Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch
